In [1]:
from shapely.geometry import Point 
from shapely.geometry import LineString
import pandas as pd
import geopandas as gpd
import sys
import matplotlib.pyplot as plt
import datetime as datetime
import numpy as np
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
from folium import plugins
from folium.plugins import HeatMap
from folium.plugins import HeatMapWithTime

# Adding csv data

In [2]:
tornado_911 = pd.read_csv('../data/911_Phone_Calls_Tornado_030320.csv')
cadd_tornado = pd.read_csv('../data/Computer_Aided_Dispatch_Data_Tornado_Incidents_030320.csv', skiprows = 3)
derecho_911 = pd.read_csv('../data/911_Phone_Calls_Derecho_050320_050420.csv')
cadd_derecho = pd.read_csv('../data/Computer_Aided_Dispatch_Data_Derecho_Incidents_050320-050420.csv', skiprows = 3)

# Fixing columns

In [3]:
tornado_911.columns = ['time', 'calltype', 'lat', 'lng', 'cell_tower_address']
derecho_911.columns = ['time', 'calltype', 'lat', 'lng', 'cell_tower_address']
cadd_tornado.columns = ['time', 'location', 'lat', 'lng', 'incident_type']
cadd_derecho.columns = ['time', 'location', 'lat', 'lng', 'incident_type']

In [4]:
tornado_911.time = pd.to_datetime(tornado_911.time)
derecho_911.time = pd.to_datetime(derecho_911.time)
cadd_tornado.time = pd.to_datetime(cadd_tornado.time)
cadd_derecho.time = pd.to_datetime(cadd_derecho.time)

In [5]:
tornado_911 = tornado_911.dropna(subset = ['lat', 'lng'])
derecho_911 = derecho_911.dropna(subset = ['lat', 'lng'])
cadd_tornado = cadd_tornado.dropna(subset = ['lat', 'lng'])
cadd_derecho = cadd_derecho.dropna(subset = ['lat', 'lng'])

In [6]:
tornado_911['geometry'] = tornado_911.apply(lambda x: Point((float(x.lng), float(x.lat))), axis=1)
derecho_911['geometry'] = derecho_911.apply(lambda x: Point((float(x.lng), float(x.lat))), axis=1)
cadd_tornado['geometry'] = cadd_tornado.apply(lambda x: Point((float(x.lng), float(x.lat))), axis=1)
cadd_derecho['geometry'] = cadd_derecho.apply(lambda x: Point((float(x.lng), float(x.lat))), axis=1)

### Check them here

In [7]:
#tornado_911.head()

In [8]:
#derecho_911.head()

In [9]:
#cadd_tornado.head()

In [10]:
#cadd_derecho.head()

# Breaking down the dataframes by time

### For Tornado 911 Calls

In [11]:
midnight_torn_calls = tornado_911[tornado_911.time < '2020-03-03 01:00:00']
one_am_torn_calls = tornado_911[(tornado_911.time > '2020-03-03 00:59:59') & (tornado_911.time < '2020-03-03 02:00:00')]
two_am_torn_calls = tornado_911[(tornado_911.time > '2020-03-03 01:59:59') & (tornado_911.time < '2020-03-03 03:00:00')]
three_am_torn_calls = tornado_911[(tornado_911.time > '2020-03-03 02:59:59') & (tornado_911.time < '2020-03-03 04:00:00')]
four_am_torn_calls = tornado_911[(tornado_911.time > '2020-03-03 03:59:59') & (tornado_911.time < '2020-03-03 05:00:00')]

### For CADD Tornado Responses

In [12]:
midnight_cadd_torn_calls = cadd_tornado[cadd_tornado.time < '2020-03-03 01:00:00']
one_am_cadd_torn_calls = cadd_tornado[(cadd_tornado.time > '2020-03-03 00:59:59') & (cadd_tornado.time < '2020-03-03 02:00:00')]
two_am_cadd_torn_calls = cadd_tornado[(cadd_tornado.time > '2020-03-03 01:59:59') & (cadd_tornado.time < '2020-03-03 03:00:00')]
three_am_cadd_torn_calls = cadd_tornado[(cadd_tornado.time > '2020-03-03 02:59:59') & (cadd_tornado.time < '2020-03-03 04:00:00')]
four_am_cadd_torn_calls = cadd_tornado[(cadd_tornado.time > '2020-03-03 03:59:59') & (cadd_tornado.time < '2020-03-03 05:00:00')]

# Adding geojson data

In [13]:
damage_points = gpd.read_file('../data/damage_points.geojson')
path_polygons = gpd.read_file('../data/path_polygons.geojson')
tornado_paths = gpd.read_file('../data/tornado_paths.geojson')

# Fixing columns

In [14]:
damage_points.stormdate = damage_points.stormdate.apply(lambda x: datetime.datetime.fromtimestamp(x / 1e3))
damage_points.surveydate = damage_points.surveydate.apply(lambda x: datetime.datetime.fromtimestamp(x / 1e3))
tornado_paths.stormdate = tornado_paths.stormdate.apply(lambda x: datetime.datetime.fromtimestamp(x / 1e3))
tornado_paths.starttime = tornado_paths.starttime.apply(lambda x: datetime.datetime.fromtimestamp(x / 1e3))
tornado_paths.endtime = tornado_paths.endtime.apply(lambda x: datetime.datetime.fromtimestamp(x / 1e3))

### Check them here

In [15]:
#damage_points.head()

In [16]:
#path_polygons.head()

In [17]:
#tornado_paths

### Categorizing damage

In [18]:
for index, damage_txt in enumerate(damage_points['damage_txt']):
    if damage_txt in ['Small Barns or Farm Outbuildings (SBO)', 'Metal Building System (MBS)', 
                     'Low-Rise Building [1-4 Stories] (LRB)', 'Electrical Transmission Lines (ETL)',
                     'Small Professional Building [Doctors Office, Branch Banks] (SPB)',
                     'Institutional Building [Hospital, Government or University Building] (IB)',
                     'Large, Isolated Retail Building [K-Mart, Wal-Mart] (LIRB)',
                     'Junior or Senior High School (JHSH)', 'Strip Mall (SM)',
                     'Warehouse Building [Tilt-up Walls or Heavy-Timber Construction] (WHB)',
                     'Elementary School [Single Story; Interior or Exterior Hallways] (ES)',
                     'Free-Standing Towers (FST)', 'Service Station Canopy (SSC)',
                     'Mid-Rise Building [5-20 Stories] (MRB)']:
        damage_points.loc[index, 'category'] = 'Commercial'
    elif damage_txt in ['One- or Two-Family Residences (FR12)', 'Masonry Apartment or Motel Building (MAM)',
                       'Manufactured Home - Single Wide (MHSW)', 'Apartments, Condos, Townhouses [3 stories or less] (ACT)',
                       'Manufactured Home - Double Wide (MHDW)', 'Motel (M)']:
        damage_points.loc[index, 'category'] = 'Residential'
    elif damage_txt in ['Trees: Hardwood (TH)', 'Trees: Softwood (TS)']:
        damage_points.loc[index, 'category'] = 'Trees'
    elif damage_txt in ['Other (O)']:
        damage_points.loc[index, 'category'] = 'Other'  

# Making GeoDataFrames

In [19]:
geo_tornado_911 = gpd.GeoDataFrame(tornado_911, crs = tornado_paths.crs, geometry = tornado_911.geometry)
geo_derecho_911 = gpd.GeoDataFrame(derecho_911, crs = tornado_paths.crs, geometry = derecho_911.geometry)
geo_cadd_tornado = gpd.GeoDataFrame(cadd_tornado, crs = tornado_paths.crs, geometry = cadd_tornado.geometry)
geo_cadd_derecho = gpd.GeoDataFrame(cadd_derecho, crs = tornado_paths.crs, geometry = cadd_derecho.geometry)

# Filtering for the Nashville Tornado

In [20]:
nashville_tornado = tornado_paths.loc[tornado_paths.event_id == 'Nashville']
nashville_path = path_polygons.geometry.iloc[np.r_[4, 10:25, 26:35]]
nashville_path = nashville_path.reset_index()
nashville_path = nashville_path.drop(columns = ['index'])
nashville_damage = gpd.sjoin(damage_points, nashville_path, how = 'inner', op = 'within')
nashville_damage = nashville_damage.drop_duplicates(keep = 'first', subset = ['objectid'])

### Check them here

In [21]:
#nashville_tornado

In [22]:
#nashville_path

In [23]:
#nashville_damage.head()

# Mapping the Nashville Tornado

### Calls

In [24]:
style = {'fillColor': 'red', 'color': 'red'}
style2 = {'fillColor': 'lightgreen', 'color': 'lightgreen'}
startloc = [36.1627, -86.7816]

map_calls = folium.Map(location = startloc, zoom_start = 10)

folium.GeoJson(nashville_tornado.geometry, style_function = lambda x:style).add_to(map_calls)
folium.GeoJson(nashville_path.geometry, style_function = lambda x:style2).add_to(map_calls)

for row_index, row_values in midnight_torn_calls.iterrows():
    loc = [row_values['lat'], row_values['lng']]
    pop = [row_values['time']]
    icon = folium.Icon(color = 'red', icon = 'phone', prefix = 'fa')
    marker = folium.Marker(location = loc, popup = pop, icon = icon)
    marker.add_to(map_calls)
    
for row_index, row_values in one_am_torn_calls.iterrows():
    loc = [row_values['lat'], row_values['lng']]
    pop = [row_values['time']]
    icon = folium.Icon(color = 'orange', icon = 'phone', prefix = 'fa')
    marker = folium.Marker(location = loc, popup = pop, icon = icon)
    marker.add_to(map_calls)
    
for row_index, row_values in two_am_torn_calls.iterrows():
    loc = [row_values['lat'], row_values['lng']]
    pop = [row_values['time']]
    icon = folium.Icon(color = 'green', icon = 'phone', prefix = 'fa')
    marker = folium.Marker(location = loc, popup = pop, icon = icon)
    marker.add_to(map_calls)
    
for row_index, row_values in three_am_torn_calls.iterrows():
    loc = [row_values['lat'], row_values['lng']]
    pop = [row_values['time']]
    icon = folium.Icon(color = 'darkblue', icon = 'phone', prefix = 'fa')
    marker = folium.Marker(location = loc, popup = pop, icon = icon)
    marker.add_to(map_calls)
    
for row_index, row_values in four_am_torn_calls.iterrows():
    loc = [row_values['lat'], row_values['lng']]
    pop = [row_values['time']]
    icon = folium.Icon(color = 'purple', icon = 'phone', prefix = 'fa')
    marker = folium.Marker(location = loc, popup = pop, icon = icon)
    marker.add_to(map_calls)
    
map_calls.save('../maps/map_calls.html')
    
map_calls

### CADD Response

In [25]:
style = {'fillColor': 'red', 'color': 'red'}
style2 = {'fillColor': 'lightgreen', 'color': 'lightgreen'}
startloc = [36.1627, -86.7816]

map_cadd_res = folium.Map(location = startloc, zoom_start = 10)

folium.GeoJson(nashville_tornado.geometry, style_function = lambda x:style).add_to(map_cadd_res)
folium.GeoJson(nashville_path.geometry, style_function = lambda x:style2).add_to(map_cadd_res)

for row_index, row_values in midnight_cadd_torn_calls.iterrows():
    loc = [row_values['lat'], row_values['lng']]
    pop = [row_values['time']]
    icon = folium.Icon(color = 'red', icon = 'ambulance', prefix = 'fa')
    marker = folium.Marker(location = loc, popup = pop, icon = icon)
    marker.add_to(map_cadd_res)
    
for row_index, row_values in one_am_cadd_torn_calls.iterrows():
    loc = [row_values['lat'], row_values['lng']]
    pop = [row_values['time']]
    icon = folium.Icon(color = 'orange', icon = 'ambulance', prefix = 'fa')
    marker = folium.Marker(location = loc, popup = pop, icon = icon)
    marker.add_to(map_cadd_res)
    
for row_index, row_values in two_am_cadd_torn_calls.iterrows():
    loc = [row_values['lat'], row_values['lng']]
    pop = [row_values['time']]
    icon = folium.Icon(color = 'green', icon = 'ambulance', prefix = 'fa')
    marker = folium.Marker(location = loc, popup = pop, icon = icon)
    marker.add_to(map_cadd_res)
    
for row_index, row_values in three_am_cadd_torn_calls.iterrows():
    loc = [row_values['lat'], row_values['lng']]
    pop = [row_values['time']]
    icon = folium.Icon(color = 'darkblue', icon = 'ambulance', prefix = 'fa')
    marker = folium.Marker(location = loc, popup = pop, icon = icon)
    marker.add_to(map_cadd_res)
    
for row_index, row_values in four_am_cadd_torn_calls.iterrows():
    loc = [row_values['lat'], row_values['lng']]
    pop = [row_values['time']]
    icon = folium.Icon(color = 'purple', icon = 'ambulance', prefix = 'fa')
    marker = folium.Marker(location = loc, popup = pop, icon = icon)
    marker.add_to(map_cadd_res)
    
map_cadd_res.save('../maps/map_cadd_res.html')
    
map_cadd_res